In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

## Classification and Imputation using Auto-Encoder

The idea is to train a dense neural network as an auto-encoder to fill in missing values.


## Outputs
Creates output files in the "predictions" directory.
Imputes as much as possible all NaNs in both train and test files and adds predictions.

**IMPORTANT**: Preidictions in the train set were created by the Auto-Encoder!!


In [2]:
# Load data
train = pd.read_csv('data/trainSet.txt')
test = pd.read_csv('data/testSet.txt')

In [3]:
# definde columns of interest
cols_out = ['PatientID', 'ImageFile', 'Hospital', 'Prognosis']
binary_vars = ['RespiratoryFailure', 'Sex', 'Cough', 'DifficultyInBreathing', 'CardiovascularDisease']
impute_cols = [i for i in train.columns if i not in cols_out]

## Autoencoder should work!

In [4]:
def create_model():
    
    regularizer = keras.regularizers.l1_l2(l1=0, l2=0.00075)
    
    inputs = keras.Input(shape=(16,))
    x = keras.layers.Dense(16, activation='selu')(inputs)
    # build a little auto-encoder
    x = keras.layers.Dense(100, activation='selu', kernel_regularizer=regularizer)(x)
    x = keras.layers.Dropout(0.5)(x)
    # bottleneck
    x = keras.layers.Dense(4, activation='selu', kernel_regularizer=regularizer)(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(100, activation='selu', kernel_regularizer=regularizer)(x)
    x = keras.layers.Dropout(0.5)(x)
    
    out1 = keras.layers.Dense(16, activation='sigmoid', name='imputing')(x)
    out2 = keras.layers.Dense(1, activation='sigmoid', name='classifying')(x)
    
    model = keras.Model(inputs, {'impute': out1, 'classify':out2})
    
    return model

In [5]:
# From https://github.com/keras-team/keras/issues/7065
# Calculate MSE only on known values
def masked_mse(mask_value=-1):
    def f(y_true, y_pred):
        mask_true = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        masked_squared_error = K.square(mask_true * (y_true - y_pred))
        masked_mse = K.sum(masked_squared_error, axis=-1) / K.sum(mask_true, axis=-1)
        return masked_mse * 100

    return f


# Validate imputation by removing one nan extra per column
def imputer_validation(valid_set):
    """
    Function to remove extra fields from data for validation.
    """
    fields_removed = valid_set.copy()
    imputer_mask = np.zeros(valid_set.shape)
    for n, row in enumerate(fields_removed):
        non_nan = np.where(row != -1)[0]
        set_nan = np.random.choice(non_nan)
        fields_removed[n, set_nan] = -1
        imputer_mask[n, set_nan] = True
    
    return fields_removed, imputer_mask


# helper to calculate scores
def calculate_score(x_true, x_pred, errorfun='mse', scaler=None):
    if scaler:
        x_true = scaler.inverse_transform(x_true.copy())
        x_pred = scaler.inverse_transform(x_pred.copy())
        x_true = np.clip(x_true, a_min=-1, a_max=None)
        x_pred = np.clip(x_pred, a_min=-1, a_max=None)
    if errorfun == 'mse':
        score = mean_squared_error(x_true, x_pred)
    elif errorfun == 'acc':
        score = accuracy_score(x_true, x_pred > 0.5)
    elif errorfun == "masked_mse":
        score = masked_mse(-1)(x_true, x_pred)
        score = np.mean(score)
    
    return score


def post_processing(original_df, predicted_values, classify, scaler=None):
    # Post processing
    nan_mask = np.isnan(original_df)
    previous_values = original_df.copy().values
    imputed_thing = np.zeros(previous_values.shape)

    # Rescale values and add to empty matrix
    if scaler is not None:
        predicted_values = MM.inverse_transform(predicted_values)
    
    imputed_thing[nan_mask] = predicted_values[nan_mask]
    # Clip negatives
    imputed_thing = np.clip(imputed_thing, a_min=0, a_max=None) 
    # Add known values
    imputed_thing[nan_mask==False] = previous_values[nan_mask==False]
    
    # Add classification and column names, binarize binary values
    df_out = pd.DataFrame(imputed_thing, columns=original_df.columns)
    df_out[binary_vars] = (df_out[binary_vars] > 0.5) * 1.0
    df_out['Prognosis'] = classify > 0.5
    
    return df_out

In [6]:
# some global vars
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
OPTIMIZER = keras.optimizers.Adam(lr=0.00075)
CALLBACKS = [early_stopping]
LOSS = {'impute': masked_mse(), 'classify': 'binary_crossentropy'}

In [7]:
# Preprare data for training
train_clean = train[impute_cols]
train_y = train['Prognosis'] == 'MILD'

In [8]:
accs, mses0, mses1, imps0, imps1, trues, preds, indices, hosp_val = [], [], [], [], [], [], [], [], []

train_mask = np.isnan(train_clean).values

for hosp in np.unique(train.Hospital):
    model = create_model()
    model.compile(optimizer=OPTIMIZER, 
                  loss=LOSS,
                  metrics={'impute': 'mse', 'classify': 'accuracy'})
    MM = MinMaxScaler(clip=True)
    
    tr = np.where(train.Hospital != hosp)[0]
    vl = np.where(train.Hospital == hosp)[0]

    # Remove Nans
    x_train = train_clean.iloc[tr].copy().fillna(0)
    x_valid = train_clean.iloc[vl].copy().fillna(0)
    # Define y-values
    y_train = train_y.iloc[tr]
    y_valid = train_y.iloc[vl]
    # Scale data from 0 to 1
    x_train = MM.fit_transform(x_train)
    x_valid = MM.transform(x_valid)
    
    # Set former nan values to -1
    x_train[train_mask[tr]] = -1
    x_valid[train_mask[vl]] = -1
    
    # Fit Model
    model.fit(x_train, {'impute':x_train, 'classify': y_train}, verbose=0, 
              callbacks=CALLBACKS, validation_split=0.15,
             epochs=250)
    
    # Predict the things
    prediction = model.predict(x_valid)
    preds.append(MM.inverse_transform(prediction['impute']))

    pred_val, pred_y = prediction['impute'], prediction['classify']
    
    
    # Returns metrics, weirdly
    _, _, _, train_acc, train_mse = model.evaluate(x_train, 
                                                   {'impute':x_train, 
                                                    'classify': y_train}, 
                                                   verbose=0)
    
    # Set previous NAN to -1
    pred_val[train_mask[vl]] = -1
    x_valid[train_mask[vl]] = -1
    
    # imputer validation:
    x_impute, impute_mask = imputer_validation(x_valid)
    
    imputer_predict = model.predict(x_impute)['impute']
    imputer_predict[impute_mask==False] = -1
    
    validation_mask = x_valid.copy()
    validation_mask[impute_mask==False] = -1
    
    # imputer error
    imp0 = calculate_score(validation_mask, 
                            imputer_predict, errorfun='masked_mse')
    imp1 = calculate_score(validation_mask, 
                            imputer_predict, errorfun='masked_mse', scaler=MM)
    
    # Keepingt track
    imps0.append(imp0)
    imps1.append(imp1)
    mses0.append(calculate_score(x_valid, pred_val, scaler=MM, errorfun='masked_mse'))
    mses1.append(calculate_score(x_valid, pred_val, errorfun='masked_mse'))
    accs.append(calculate_score(y_valid, pred_y, errorfun='acc'))
    trues.append(MM.inverse_transform(x_valid))
    preds.append(MM.inverse_transform(prediction['impute']))
    print(f"Split {hosp}, with validation n = {x_valid.shape[0]}: \nRescaled MSE - {mses0[-1]:4.2f} " +
          f"MSE - {mses1[-1]:4.2f} " +
          f"Acc - {accs[-1]:4.2f}\n" +
          f"Impute MSE - {imps0[-1]:4.2f} " +
          f"Impute MSE scaled - {imps1[-1]:4.2f}")
          
    print(f"Train - MSE {train_mse:4.3f}, ACC {train_acc:4.3f}")
    
    indices.append(vl)
    hosp_val.append([hosp] * len(vl))
    print("==============================================")

Split B, with validation n = 104: 
Rescaled MSE - 80719576.00 MSE - 1.17 Acc - 0.69
Impute MSE - 14.17 Impute MSE scaled - 412857472.00
Train - MSE 0.556, ACC 0.652
Split C, with validation n = 31: 
Rescaled MSE - 8185430.50 MSE - 1.21 Acc - 0.55
Impute MSE - 19.65 Impute MSE scaled - 5366.02
Train - MSE 0.518, ACC 0.665
Split D, with validation n = 139: 
Rescaled MSE - 116832936.00 MSE - 2.18 Acc - 0.71
Impute MSE - 18.01 Impute MSE scaled - 778023872.00
Train - MSE 0.552, ACC 0.633
Split E, with validation n = 101: 
Rescaled MSE - 163067.19 MSE - 0.93 Acc - 0.65
Impute MSE - 12.30 Impute MSE scaled - 103317.16
Train - MSE 0.524, ACC 0.646
Split F, with validation n = 488: 
Rescaled MSE - 91435520.00 MSE - 2.47 Acc - 0.59
Impute MSE - 13.81 Impute MSE scaled - 239072352.00
Train - MSE 0.491, ACC 0.685


# Final Training and Creating Test Set

In [9]:
model = create_model()
model.compile(optimizer=OPTIMIZER, 
                  loss=LOSS,
                  metrics={'impute': 'mae', 'classify': 'accuracy'})

MM = MinMaxScaler(clip=True)

# Transform Data:
x_train = train_clean.copy().fillna(0)
# Define y-values
y_train = train_y
# Scale data from 0 to 1
x_train = MM.fit_transform(x_train)
# Set former nan values to -1
x_train[train_mask] = -1


model.fit(x_train, {'impute':x_train, 'classify': y_train}, verbose=1, callbacks=CALLBACKS, 
          validation_split=0.15, epochs=250)

Epoch 1/250
23/23 [==============================] - 0s 13ms/step - loss: 14.5813 - classifying_loss: 0.7806 - imputing_loss: 13.7690 - classifying_accuracy: 0.4802 - imputing_mae: 0.5492 - val_loss: 9.0251 - val_classifying_loss: 0.7162 - val_imputing_loss: 8.2783 - val_classifying_accuracy: 0.4385 - val_imputing_mae: 0.5059
Epoch 2/250
23/23 [==============================] - 0s 3ms/step - loss: 10.1582 - classifying_loss: 0.7374 - imputing_loss: 9.3910 - classifying_accuracy: 0.5061 - imputing_mae: 0.4881 - val_loss: 7.6607 - val_classifying_loss: 0.6655 - val_imputing_loss: 6.9661 - val_classifying_accuracy: 0.5692 - val_imputing_mae: 0.4920
Epoch 3/250
23/23 [==============================] - 0s 3ms/step - loss: 9.3970 - classifying_loss: 0.7305 - imputing_loss: 8.6380 - classifying_accuracy: 0.5130 - imputing_mae: 0.4761 - val_loss: 7.3992 - val_classifying_loss: 0.6776 - val_imputing_loss: 6.6938 - val_classifying_accuracy: 0.5923 - val_imputing_mae: 0.4854
Epoch 4/250
23/23 [==

23/23 [==============================] - 0s 3ms/step - loss: 5.8827 - classifying_loss: 0.6814 - imputing_loss: 5.1791 - classifying_accuracy: 0.5757 - imputing_mae: 0.4219 - val_loss: 3.7579 - val_classifying_loss: 0.6571 - val_imputing_loss: 3.0786 - val_classifying_accuracy: 0.6308 - val_imputing_mae: 0.4382
Epoch 27/250
23/23 [==============================] - 0s 3ms/step - loss: 5.9017 - classifying_loss: 0.6806 - imputing_loss: 5.1990 - classifying_accuracy: 0.5907 - imputing_mae: 0.4220 - val_loss: 3.7414 - val_classifying_loss: 0.6597 - val_imputing_loss: 3.0597 - val_classifying_accuracy: 0.6308 - val_imputing_mae: 0.4378
Epoch 28/250
23/23 [==============================] - 0s 3ms/step - loss: 5.6938 - classifying_loss: 0.6754 - imputing_loss: 4.9965 - classifying_accuracy: 0.5989 - imputing_mae: 0.4189 - val_loss: 3.7466 - val_classifying_loss: 0.6623 - val_imputing_loss: 3.0624 - val_classifying_accuracy: 0.6154 - val_imputing_mae: 0.4378
Epoch 29/250
23/23 [===============

23/23 [==============================] - 0s 3ms/step - loss: 5.1723 - classifying_loss: 0.6723 - imputing_loss: 4.4794 - classifying_accuracy: 0.5866 - imputing_mae: 0.4111 - val_loss: 3.3311 - val_classifying_loss: 0.6519 - val_imputing_loss: 2.6585 - val_classifying_accuracy: 0.6538 - val_imputing_mae: 0.4299
Epoch 52/250
23/23 [==============================] - 0s 3ms/step - loss: 5.4354 - classifying_loss: 0.6705 - imputing_loss: 4.7442 - classifying_accuracy: 0.6003 - imputing_mae: 0.4134 - val_loss: 3.2269 - val_classifying_loss: 0.6512 - val_imputing_loss: 2.5550 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4286
Epoch 53/250
23/23 [==============================] - 0s 3ms/step - loss: 5.3781 - classifying_loss: 0.6720 - imputing_loss: 4.6853 - classifying_accuracy: 0.5866 - imputing_mae: 0.4126 - val_loss: 3.2288 - val_classifying_loss: 0.6542 - val_imputing_loss: 2.5538 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4283
Epoch 54/250
23/23 [===============

23/23 [==============================] - 0s 5ms/step - loss: 5.3901 - classifying_loss: 0.6738 - imputing_loss: 4.6960 - classifying_accuracy: 0.6085 - imputing_mae: 0.4119 - val_loss: 3.1256 - val_classifying_loss: 0.6531 - val_imputing_loss: 2.4522 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4248
Epoch 77/250
23/23 [==============================] - 0s 5ms/step - loss: 5.3406 - classifying_loss: 0.6663 - imputing_loss: 4.6540 - classifying_accuracy: 0.5907 - imputing_mae: 0.4107 - val_loss: 3.0736 - val_classifying_loss: 0.6488 - val_imputing_loss: 2.4046 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4244
Epoch 78/250
23/23 [==============================] - 0s 5ms/step - loss: 5.4989 - classifying_loss: 0.6662 - imputing_loss: 4.8125 - classifying_accuracy: 0.6112 - imputing_mae: 0.4134 - val_loss: 3.0930 - val_classifying_loss: 0.6489 - val_imputing_loss: 2.4238 - val_classifying_accuracy: 0.6615 - val_imputing_mae: 0.4247
Epoch 79/250
23/23 [===============

23/23 [==============================] - 0s 5ms/step - loss: 5.1890 - classifying_loss: 0.6733 - imputing_loss: 4.4945 - classifying_accuracy: 0.5962 - imputing_mae: 0.4103 - val_loss: 2.8617 - val_classifying_loss: 0.6480 - val_imputing_loss: 2.1925 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4191
Epoch 101/250
23/23 [==============================] - 0s 4ms/step - loss: 5.2061 - classifying_loss: 0.6833 - imputing_loss: 4.5015 - classifying_accuracy: 0.5812 - imputing_mae: 0.4082 - val_loss: 2.9052 - val_classifying_loss: 0.6571 - val_imputing_loss: 2.2267 - val_classifying_accuracy: 0.6538 - val_imputing_mae: 0.4193
Epoch 102/250
23/23 [==============================] - 0s 5ms/step - loss: 5.1696 - classifying_loss: 0.6628 - imputing_loss: 4.4855 - classifying_accuracy: 0.6003 - imputing_mae: 0.4090 - val_loss: 2.8737 - val_classifying_loss: 0.6527 - val_imputing_loss: 2.1997 - val_classifying_accuracy: 0.6385 - val_imputing_mae: 0.4191
Epoch 103/250
23/23 [============

23/23 [==============================] - 0s 5ms/step - loss: 5.0690 - classifying_loss: 0.6695 - imputing_loss: 4.3774 - classifying_accuracy: 0.6016 - imputing_mae: 0.4065 - val_loss: 2.6415 - val_classifying_loss: 0.6491 - val_imputing_loss: 1.9703 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4145
Epoch 126/250
23/23 [==============================] - 0s 5ms/step - loss: 5.1969 - classifying_loss: 0.6825 - imputing_loss: 4.4922 - classifying_accuracy: 0.5921 - imputing_mae: 0.4078 - val_loss: 2.6989 - val_classifying_loss: 0.6578 - val_imputing_loss: 2.0189 - val_classifying_accuracy: 0.6462 - val_imputing_mae: 0.4157
Epoch 127/250
23/23 [==============================] - 0s 5ms/step - loss: 5.0333 - classifying_loss: 0.6750 - imputing_loss: 4.3360 - classifying_accuracy: 0.5935 - imputing_mae: 0.4066 - val_loss: 2.6623 - val_classifying_loss: 0.6639 - val_imputing_loss: 1.9761 - val_classifying_accuracy: 0.6154 - val_imputing_mae: 0.4141
Epoch 128/250
23/23 [============

23/23 [==============================] - 0s 5ms/step - loss: 5.0835 - classifying_loss: 0.6827 - imputing_loss: 4.3770 - classifying_accuracy: 0.5771 - imputing_mae: 0.4065 - val_loss: 2.4968 - val_classifying_loss: 0.6538 - val_imputing_loss: 1.8192 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4108
Epoch 151/250
23/23 [==============================] - 0s 5ms/step - loss: 4.8957 - classifying_loss: 0.6637 - imputing_loss: 4.2081 - classifying_accuracy: 0.6085 - imputing_mae: 0.4031 - val_loss: 2.4857 - val_classifying_loss: 0.6462 - val_imputing_loss: 1.8157 - val_classifying_accuracy: 0.6692 - val_imputing_mae: 0.4104
Epoch 152/250
23/23 [==============================] - 0s 5ms/step - loss: 4.9587 - classifying_loss: 0.6711 - imputing_loss: 4.2637 - classifying_accuracy: 0.5798 - imputing_mae: 0.4049 - val_loss: 2.4835 - val_classifying_loss: 0.6444 - val_imputing_loss: 1.8151 - val_classifying_accuracy: 0.6769 - val_imputing_mae: 0.4105
Epoch 153/250
23/23 [============

In [10]:
train_predictions = model.predict(x_train)
train_imputations = train_predictions['impute']
train_classify = train_predictions['classify']

train_out = post_processing(train_clean, train_imputations, train_classify, MM)

In [11]:
# Add missing columns
train_out[['PatientID', 'ImageFile', 'Hospital']] = train[['PatientID', 'ImageFile', 'Hospital']]
# Change order to original
train_out = train_out[train.columns]

train_out.to_csv('predictions/simon_TRAIN_impute_classify.csv', index=None)

In [12]:
test_clean = test[impute_cols] 
x_test = test_clean.copy().fillna(0)
x_test = MM.transform(x_test)
x_test[np.isnan(test_clean)] = -1

In [13]:
predictions = model.predict(x_test)

In [14]:
# Predidctions
imputations = predictions['impute']
classify = predictions['classify']

In [15]:
test_out = post_processing(test_clean, imputations, classify, MM)

test_out.loc[test_out.Prognosis==True, 'Prognosis'] = "MILD"
test_out.loc[test_out.Prognosis==False, 'Prognosis'] = "SEVERE"

In [16]:
# Add missing columns
test_out[['PatientID', 'ImageFile', 'Hospital']] = test[['PatientID', 'ImageFile', 'Hospital']]
# Change order to original
test_out = test_out[test.columns]

In [17]:
# Save to file
test_out.to_csv('predictions/simon_TEST_impute_classify.csv', index=False)